In [1]:
import requests
import pandas as pd
import numpy as np
import sqlalchemy
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from pandas_market_calendars import get_calendar
from dateutil.relativedelta import relativedelta

In [3]:
engine = sqlalchemy.create_engine("sqlite:///C:/Users/lewis/OneDrive/tradingstrategies/databases/momentum_strategy_2_database.db")
calendar = get_calendar("NYSE")
polygon_api_key = "KkfCQ7fsZnx0yK4bhX9fD81QplTh0Pf3"

In [4]:
stock_returns = pd.read_sql("SELECT * FROM stock_returns", con = engine)
stock_returns = stock_returns.set_index('index')


In [6]:
start_date_long = (pd.to_datetime(stock_returns.index[0]).date())
end_date = pd.to_datetime(stock_returns.index[-1]).date()
dates_long = calendar.schedule(start_date = start_date_long, end_date = end_date).index.strftime("%Y-%m-%d").values
start_date_short = pd.to_datetime(dates_long[30]).date()
dates_short = calendar.schedule(start_date = start_date_short, end_date = end_date).index.strftime("%Y-%m-%d").values


In [7]:
portfolio_returns = pd.DataFrame(columns=['Date', 'Long Momentum Returns', 'Short Momentum Returns', 'Gross Returns'])
times = []
portfolios = pd.DataFrame(columns=['Date', 'Long Portfolio', 'Short Portfolio'])
summary_stats = pd.DataFrame(columns=['Long Return', 'Short Return', 'L/S Return', 'Benchmark Return', 'Long Std', 'Short Std', 'L/S Std', 'Benchmark Std', 'L/S Benchmark Corr'])


In [38]:
yday = dates_short[-2]

In [39]:
screening_df = pd.DataFrame()
screening_df['ticker'] = stock_returns.columns
screening_df['t momentum'] = screening_df['ticker'].map(stock_returns.loc[dates_long[np.where(dates_long == yday)[0][0]-9]:yday, :].cumprod(axis=0).iloc[-1,:]-1)
screening_df['t-1 momentum'] = screening_df['ticker'].map(stock_returns.loc[dates_long[np.where(dates_long == yday)[0][0]-19]:dates_long[np.where(dates_long == yday)[0][0]-10], :].cumprod(axis=0).iloc[-1,:]-1)
screening_df['t-2 momentum'] = screening_df['ticker'].map(stock_returns.loc[dates_long[np.where(dates_long == yday)[0][0]-29]:dates_long[np.where(dates_long == yday)[0][0]-20], :].cumprod(axis=0).iloc[-1,:]-1)

In [40]:
 #adm = average daily momentum
screening_df['t adm'] = screening_df['t momentum'] / 10
screening_df['t-1 adm'] = screening_df['t-1 momentum'] / 10
screening_df['t-2 adm'] = screening_df['t-2 momentum'] / 10
#ddm = difference in daily momentum
screening_df['t/t-1 ddm'] = screening_df['t adm'] - screening_df['t-1 adm']
screening_df['t-1/t-2 ddm'] = screening_df['t-1 adm'] - screening_df['t-2 adm']

In [41]:
increasing_momentum_conditions = (screening_df['t adm'] > screening_df['t-1 adm']) & (screening_df['t-1 adm'] > screening_df['t-2 adm']) & (screening_df['t/t-1 ddm'] > screening_df['t-1/t-2 ddm'])
decreasing_momentum_conditions = (screening_df['t adm'] < screening_df['t-1 adm']) & (screening_df['t-1 adm'] < screening_df['t-2 adm']) & (screening_df['t/t-1 ddm'] < screening_df['t-1/t-2 ddm'])

In [42]:
long_momentum_portfolio = screening_df[increasing_momentum_conditions]['ticker'].tolist()
short_momentum_portfolio = screening_df[decreasing_momentum_conditions]['ticker'].tolist()

In [67]:

benchmark_data = pd.read_sql("SELECT * FROM benchmark_data", con = engine)
benchmark_data = benchmark_data.set_index('t')
benchmark_data = benchmark_data.loc[dates_short[-10]:]

In [68]:
benchmark_data

,Benchmark Price,Benchmark Return
t,,
2024-08-15,553.07,1.017140
2024-08-16,554.31,1.002242
2024-08-19,559.61,1.009561
2024-08-20,558.70,0.998374
2024-08-21,560.62,1.003437
2024-08-22,556.22,0.992152
2024-08-23,562.13,1.010625
2024-08-26,560.79,0.997616
2024-08-27,561.56,1.001373


In [43]:
long_return = (stock_returns[stock_returns.columns.isin(long_momentum_portfolio)][dates_short[-1]]).mean()

ValueError: Item wrong length 1000 instead of 2517.